In [1]:
import pandas as pd
import numpy as np
import math
import random

In [12]:
master_data = pd.read_csv("../../Data/filtered/master-data.csv", 
                          sep=",", 
                          encoding='Latin-1', 
                          low_memory=False)

In [13]:
master_data.head()

,compensation,year,country,age,gender,IT_experience_in_years,company_size,occupation,proficient_languages,desktop_OS,job_satisfaction
0,Student / Unemployed,2011,Africa,< 20,female,<2,Start Up (1-25),Web Application Developer,['JavaScript'],Linux,FML
1,NaN,2011,Other Europe,25-29,male,41310,Mature Small Business (25-100),Server Programmer,"['SQL', 'C']",Windows 7,So happy it hurts
2,NaN,2011,India,25-29,female,41435,Mid Sized (100-999),Server Programmer,"['JavaScript', 'SQL']",Linux,NaN
3,Student / Unemployed,2011,Germany,< 20,female,41310,Student,Student,['Haskell'],Linux,I enjoy going to work
4,"$80,000 - $100,000",2011,Other Asia,35-39,male,11,Start Up (1-25),"Executive (VP of Eng, CTO, CIO, etc.)","['JavaScript', 'CSS', 'PHP', 'SQL', 'C++', 'C'...",Linux,It pays the bills


In [4]:
master_data.age.unique()

array(['< 20', '25-29', '35-39', '20-24', nan, '30-34', '51-60', '40-50',
       '>60', 'Prefer not to disclose', '> 60', '40-49', '50-59', '24.0',
       '31.0', '42.0', '36.0', '28.0', '32.0', '35.0', '34.0', '37.0',
       '33.0', '30.0', '27.0', '40.0', '29.0', '23.0', '25.0', '26.0',
       '39.0', '38.0', '41.0', '25 - 34 years old', '35 - 44 years old',
       '18 - 24 years old', '45 - 54 years old', '55 - 64 years old',
       'Under 18 years old', '65 years or older', '14.0', '19.0', '22.0',
       '21.0', '20.0', '47.0', '17.0', '44.0', '43.0', '62.0', '45.0',
       '18.0', '16.0', '64.0', '54.0', '49.0', '56.0', '12.0', '58.0',
       '46.0', '59.0', '51.0', '48.0', '57.0', '52.0', '50.0', '23.9',
       '55.0', '15.0', '67.0', '13.0', '1.0', '53.0', '69.0', '65.0',
       '17.5', '63.0', '61.0', '68.0', '73.0', '70.0', '60.0', '16.5',
       '46.5', '11.0', '71.0', '3.0', '97.0', '29.5', '77.0', '74.0',
       '26.5', '26.3', '24.5', '78.0', '72.0', '66.0', '76.0', '10.0'

In [35]:
master_data["age"] = master_data["age"].fillna('age_unknown')

In [49]:
def normalize_gender(row):
    age = row
    if row in ["< 20", "Under 18 years old", "below 18"]:
        age="below_18"
    elif row in ["18 - 24 years old", "18-24 years old", "20-24"]:
        age="18_to_24"
    elif row in ["25 - 34 years old", "25-34 years old", "25-29", "30-34", "279.0"]:
        age="25_to_34"
    elif row in ["35 - 44 years old", "35-44 years old", "35-39"]:
        age="35_to_44"
    elif row in ["40-49", "40-50", "45 - 54 years old", "45-54 years old"]:
        age="45_to_54"
    elif row in ["50-59", "51-60", "55 - 64 years old" , "55-64 years old"]:
        age="55_to_64"
    elif row in ["65 years or older", "> 60", ">60"]:
        age="above_65"
    elif row in ["Prefer not to disclose", "Prefer not to say"]:
        age="age_unknown"
    return age

def categorise_gender(row):
    age = row 
    exclusion_list = ["age_unknown",
        "Prefer not to disclose", 
                        "Prefer not to say", 
                        "above_65", 
                        "55_to_64", 
                        "45_to_54", 
                        "35_to_44", 
                        "25_to_34", 
                        "18_to_24", 
                        "below_18"]
    if row not in exclusion_list:
        age_i = int(float(row))
        if age_i <= 18:
            age="below_18"
        elif age_i > 18 and age_i <=24:
            age="18_to_24"
        elif age_i > 24 and age_i <=34:
            age="25_to_34"
        elif age_i > 34 and age_i <=44:
            age="35_to_44"
        elif age_i > 44 and age_i <=54:
            age="45_to_54"
        elif age_i > 54 and age_i <=64:
            age="55_to_64"
        elif age_i > 64:
            age="above_65"
            
    return age

In [50]:
master_data["age"] = master_data["age"].apply(normalize_gender)

In [51]:
master_data["age"] = master_data["age"].apply(categorise_gender)

In [55]:
master_data.groupby(["year", "age"])["age"].count()

year  age        
2011  18_to_24         507
      25_to_34        1443
      35_to_44         343
      45_to_54         277
      55_to_64          37
      above_65           6
      age_unknown       84
      below_18         116
2012  18_to_24        1474
      25_to_34        3052
      35_to_44         558
      45_to_54         426
      55_to_64          77
      above_65          30
      age_unknown      279
      below_18         347
2013  18_to_24        2114
      25_to_34        4490
      35_to_44         990
      45_to_54         915
      55_to_64         260
      above_65          48
      age_unknown      308
      below_18         617
2014  18_to_24        1864
      25_to_34        3292
      35_to_44         710
      45_to_54         644
      55_to_64         166
      above_65          32
      age_unknown      297
      below_18         638
2015  18_to_24        6339
      25_to_34       11958
      35_to_44        2357
      45_to_54        1965
      55_t

In [59]:
master_data = master_data[master_data.age != 'age_unknown']

In [60]:
master_data.groupby(["year", "age"])["age"].count()

year  age     
2011  18_to_24      507
      25_to_34     1443
      35_to_44      343
      45_to_54      277
      55_to_64       37
      above_65        6
      below_18      116
2012  18_to_24     1474
      25_to_34     3052
      35_to_44      558
      45_to_54      426
      55_to_64       77
      above_65       30
      below_18      347
2013  18_to_24     2114
      25_to_34     4490
      35_to_44      990
      45_to_54      915
      55_to_64      260
      above_65       48
      below_18      617
2014  18_to_24     1864
      25_to_34     3292
      35_to_44      710
      45_to_54      644
      55_to_64      166
      above_65       32
      below_18      638
2015  18_to_24     6339
      25_to_34    11958
      35_to_44     2357
      45_to_54     1965
      55_to_64      488
      above_65      124
      below_18     2261
2016  18_to_24    13055
      25_to_34    25720
      35_to_44     5657
      45_to_54     4905
      55_to_64     1640
      above_65      422
 

In [61]:
# Writes the normalized data for data analysis
master_data.to_csv("../../Data/filtered/master-data.csv", index=False)